In [ ]:
import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine,text
import pandas as pd

In [ ]:
import sqlalchemy
sqlalchemy.__version__

'2.0.5.post1'

In [ ]:
# Function to extract table to a pandas DataFrame
def extract_table_to_pandas(tablename, db_engine):
    query = "SELECT * FROM {}".format(tablename)
    return pd.read_sql(query, db_engine)

db_engine = create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                                   format('root', '1234',
                                                          'localhost', 'elearning'))

In [ ]:
def print_user_comparison(user1, user2, user3):
    print("Course id overlap between users:")
    print("================================")
    print("User 1 and User 2 overlap: {}".format(
    set(user1.course_id) & set(user2.course_id)
    ))
    print("User 1 and User 3 overlap: {}".format(
    set(user1.course_id) & set(user3.course_id)
    ))
    print("User 2 and User 3 overlap: {}".format(
    set(user2.course_id) & set(user3.course_id)
    ))

In [ ]:
# Get user with id 4387
user1 = pd.read_sql_query(sql=text("SELECT * FROM rating where user_id=4387"),con=db_engine.connect())

# Get user with id 18163
user2 = pd.read_sql_query(sql=text("SELECT * FROM rating where user_id=18163"),con=db_engine.connect())

# Get user with id 8770
user3 = pd.read_sql(sql=text("SELECT * FROM rating where user_id=8770"),con=db_engine.connect())

# Use the helper function to compare the 3 users
print_user_comparison(user1, user2, user3)

Course id overlap between users:
User 1 and User 2 overlap: {32, 96, 36, 6, 7, 44, 95}
User 1 and User 3 overlap: set()
User 2 and User 3 overlap: set()


In [ ]:
def extract_rating_data(db_engine):
    return pd.read_sql_query(sql=text("SELECT * FROM rating"),con=db_engine.connect())

def extract_course_data(db_engine):
    return pd.read_sql_query(sql=text("SELECT * FROM courses"),con=db_engine.connect())

In [ ]:
ratings = extract_rating_data(db_engine)

course_data = extract_course_data(db_engine)

In [ ]:
ratings.head()

,user_id,course_id,rating
0,1,6,4
1,1,36,5
2,1,37,5
3,1,45,5
4,1,50,5


In [ ]:
course_data.head()

,course_id,title,description,programming_language,difficulty
0,1,Machine Learning with Apache Spark,"Spark is a powerful, general purpose tool for ...",python,hard
1,2,Financial Analytics in Spreadsheets,Monitoring the evolution of traded assets is k...,spreadsheets,hard
2,3,Intermediate R,The intermediate R course is the logical next ...,r,hard
3,4,Data Visualization with ggplot2 (Part 2),This ggplot2 tutorial builds on your knowledge...,r,hard
4,5,Fraud Detection in R,The Association of Certified Fraud Examiners e...,r,hard


In [ ]:
ratings.user_id.nunique()

38174

In [ ]:
course_data.shape

(99, 5)

In [ ]:
# Print out the number of missing values per column
print(course_data.isnull().sum())

course_id               0
title                   0
description             0
programming_language    3
difficulty              0
dtype: int64


In [ ]:
# The transformation should fill in the missing values
def transform_fill_programming_language(course_data):
    imputed = course_data.fillna({"programming_language": "R"})
    return imputed

transformed = transform_fill_programming_language(course_data)

# Print out the number of missing values per column of transformed
print(transformed.isnull().sum())

course_id               0
title                   0
description             0
programming_language    0
difficulty              0
dtype: int64


In [ ]:
# transformation function
def transform_avg_rating(rating_data):
    # Group by course_id and extract average rating per course
    avg_rating = rating_data.groupby('course_id').rating.mean()
    # Return sorted average ratings per course
    sort_rating = avg_rating.sort_values(ascending=False).reset_index()
    return sort_rating

In [ ]:
# Use transform_avg_rating on the extracted data and print results
avg_course_ratings = transform_avg_rating(ratings)
print(avg_course_ratings)

    course_id    rating
0          46  4.800000
1          23  4.800000
2          96  4.692765
3          56  4.661765
4          24  4.653061
..        ...       ...
94         54  4.238095
95         92  4.222222
96         29  4.208333
97         17  4.147059
98         42  4.107570

[99 rows x 2 columns]


In [ ]:
def get_courses_to_recommend(ratings, course_data):
    user_list = ratings.user_id.unique()

    course_set = set(course_data.course_id.unique())

    non_enrolled_users = []

    for user in user_list:
        rated_course_set = set(ratings[ratings.user_id == user].course_id.unique())
        non_enrolled_courses = course_set - rated_course_set
        non_enrolled_users = non_enrolled_users + [(user, cid) for cid in non_enrolled_courses]

    non_enrolled_courses_df = pd.DataFrame(non_enrolled_users, columns = ['user_id', 'course_id'])
    return non_enrolled_courses_df

courses_to_recommend = get_courses_to_recommend(ratings, course_data)
courses_to_recommend.to_csv('courses_to_recommend.csv', index = False)
courses_to_recommend.sample(n=500).to_csv('courses_to_recommend_small.csv', index = False)

In [ ]:
course_data.shape

(99, 5)

In [ ]:
temp = pd.read_csv('C:/Users/clash/Downloads/courses_to_recommend_small.csv')
temp.sample(n=2000,replace=True).to_csv('courses_to_recommend_small.csv', index = False)

In [ ]:
temp.shape

(500, 2)

In [ ]:
courses_to_recommend = pd.read_csv('courses_to_recommend_small.csv')

courses_to_recommend.head()

,user_id,course_id
0,24095,63
1,31649,95
2,21349,90
3,16595,75
4,2646,44


In [ ]:
courses_to_recommend.groupby('user_id').count()

,course_id
user_id,
4,1
37,3
139,8
247,1
332,6
...,...
37962,2
37993,4
38026,3


In [ ]:
# Complete the transformation function
def transform_recommendations(avg_course_ratings, courses_to_recommend):
    # Merge both DataFrames
    merged = courses_to_recommend.merge(avg_course_ratings)

    # Sort values by rating and group by user_id
    grouped = merged.sort_values("rating", ascending=False).groupby("user_id")

    # Produce the top 3 values and sort by user_id
    recommendations = grouped.head(5).sort_values("user_id").reset_index()

    final_recommendations = recommendations[["user_id", "course_id","rating"]]

    # Return final recommendations
    return final_recommendations


# Use the function with the predefined DataFrame objects
recommendations = transform_recommendations(avg_course_ratings, courses_to_recommend)

In [ ]:
recommendations.head()

,user_id,course_id,rating
0,4,92,4.222222
1,37,12,4.340909
2,37,12,4.340909
3,37,12,4.340909
4,139,9,4.470085


In [ ]:
# Merge both DataFrames
merged = courses_to_recommend.merge(avg_course_ratings)

merged.head()

,user_id,course_id,rating
0,24095,63,4.52381
1,2244,63,4.52381
2,20357,63,4.52381
3,31865,63,4.52381
4,20357,63,4.52381


In [ ]:
merged.sort_values("rating", ascending=False).head()

,user_id,course_id,rating
230,16400,46,4.8
224,17972,46,4.8
238,28036,46,4.8
237,16400,46,4.8
236,30833,46,4.8


In [ ]:
merged[merged.user_id == 30251]

,user_id,course_id,rating


In [ ]:
# Sort values by rating and group by user_id
grouped = merged.sort_values("rating", ascending=False).groupby("user_id")


# Produce the top 3 values and sort by user_id
recommendations = grouped.head(3).sort_values("user_id")

recommendations.head()

,user_id,course_id,rating
1764,4,92,4.222222
439,37,12,4.340909
438,37,12,4.340909
437,37,12,4.340909
273,139,9,4.470085


In [ ]:
recommendations[recommendations.user_id == 16456]

,user_id,course_id,rating


In [ ]:
recommendations = transform_recommendations(avg_course_ratings, courses_to_recommend)

In [ ]:
db_engine = create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                                   format('root', '1234',
                                                          'localhost', 'elearning'))

def load_to_dwh(recommendations, db_engine):
    recommendations.to_sql("recommendations", db_engine, if_exists="replace")

In [ ]:
load_to_dwh(recommendations, db_engine)

In [ ]:
def recommendations_for_user(user_id, threshold=4.5):
    # Join with the courses table
    query = """
    SELECT title, rating FROM recommendations
    INNER JOIN courses ON courses.course_id = recommendations.course_id
    WHERE user_id=%s AND rating>%s
    ORDER BY rating DESC
    """%(user_id,threshold)
    # Add the threshold parameter
    predictions_df = pd.read_sql_query(sql=text(query),con=db_engine.connect(), params = {"user_id": user_id,
                                                           "threshold": threshold})
    return predictions_df.title.values

# Try the function you created
print(recommendations_for_user(4487, 4.0))

['Network Analysis in Python (Part 1)'
 'Network Analysis in Python (Part 1)'
 'Network Analysis in Python (Part 1)'
 'Network Analysis in Python (Part 1)'
 'Network Analysis in Python (Part 1)']


## ETL: Combine all the operations

In [ ]:
def etl(db_engine):
    # Extract the data
    courses = extract_course_data(db_engine)
    rating = extract_rating_data(db_engine)

    # Clean up courses data
    courses = transform_fill_programming_language(courses)

    # Get the average course ratings
    avg_course_rating = transform_avg_rating(rating)

    # Get eligible user and course id pairs
    courses_to_recommend = pd.read_csv('courses_to_recommend_small.csv')

    # Calculate the recommendations
    recommendations = transform_recommendations(
    avg_course_rating,
    courses_to_recommend,
    )

    # Load the recommendations into the database
    load_to_dwh(recommendations, db_engine)

In [ ]:
etl(db_engine)

In [ ]:
# i have introduced a new column named difficulty in courses table containing easy,medium and hard as difficulty for the courses
print("how do you want to get courses recommended")
print("type 1 for recommendation on the basis of programming language")
print("type 2 for recommendation on the basis of difficulty")
a=int(input("enter your choice "))
if a==1:
    name=input("enter the programming language name as r,python,sql,shell,spreadsheets ")
    print("\t The courses that we recommend to you are \t")
    data=pd.read_sql_query(sql=text("select title,description from courses where programming_language='%s'"%(name)),con=db_engine.connect())
    print(data["title"])
    desc=list(map(int,input("enter the course numbers of which you want description seperated by space ").split(" ")))
    print(data.loc[desc])
if a==2:
    print("you are a begineer,intermediate or professional")
    print("type 1 for begineer")
    print("type 2 for intermediate")
    print("type 3 for professional")
    n=int(input("enter your choice "))
    if n==1:
        print("\t The courses that we recommend to you are \t")
        data=pd.read_sql_query(sql=text("select title,description from courses where difficulty='easy'"),con=db_engine.connect())
        print(data["title"])
        desc=list(map(int,input("enter the course numbers of which you want description seperated by space ").split(" ")))
        print(data.loc[desc])
    elif n==2:
        print("\t The courses that we recommend to you are \t")
        data=pd.read_sql_query(sql=text("select title,description from courses where difficulty='medium'"),con=db_engine.connect())
        print(data["title"])
        desc=list(map(int,input("enter the course numbers of which you want description seperated by space ").split(" ")))
        print(data.loc[desc])
    elif n==3:
        print("\t The courses that we recommend to you are \t")
        data=pd.read_sql_query(sql=text("select title,description from courses where difficulty='hard'"),con=db_engine.connect())
        print(data["title"])
        desc=list(map(int,input("enter the course numbers of which you want description seperated by space ").split(" ")))
        print(data.loc[desc])
else:
    print("invalid choice")

how do you want to get courses recommended
type 1 for recommendation on the basis of programming language
type 2 for recommendation on the basis of difficulty
enter your choice 2
you are a begineer,intermediate or professional
type 1 for begineer
type 2 for intermediate
type 3 for professional
enter your choice 1
	 The courses that we recommend to you are 	
0                                Introduction to Python
1                             Credit Risk Modeling in R
2                     Visualizing Time Series Data in R
3                                 Equity Valuation in R
4                    Data-Driven Decision Making in SQL
5        Intermediate Functional Programming with purrr
6          Importing & Cleaning Data in R: Case Studies
7            Bayesian Regression Modeling with rstanarm
8                  Intermediate Python for Data Science
9            Interactive Data Visualization with rbokeh
10                              Defensive R Programming
11                      

Flow chart
![1.png](attachment:1.png)


![2.png](attachment:2.png)